In [1]:
import os
import numpy as np
import pandas as pd

# load data '../../chars.csv'
chars = pd.read_csv('../characters_freq.csv', sep=',', encoding='utf-8')
# chars = chars[chars['level'] == '一级']

words = pd.read_csv('../vocabulary_freq.csv', sep=',', encoding='utf-8')
# words = words[words['level'] == '一级']

chars.head()

,index,level,character,pinyin
0,7.79,一级,的,de
1,7.79,四级,的,dí
2,7.79,二级,的,dì
3,7.17,一级,是,shì
4,7.16,一级,在,zài


In [2]:


# new dataframe
df = pd.DataFrame(columns=['page', 'chars', 'words', 'level', 'pinyin'])

page = 0
for index, row in chars.iterrows():

    page += 1

    df.loc[index, 'page'] = page
    df.loc[index, 'chars'] = row['character']
    df.loc[index, 'level'] = row['level']
    df.loc[index, 'pinyin'] = row['pinyin']

In [3]:
df['old_chars'] = ''
for page, row in df.iterrows():
    cum_chars = df.loc[:page, 'chars'].str.cat(sep=', ')
    df.at[page, 'old_chars'] = cum_chars
df

,page,chars,words,level,pinyin,old_chars
0,1,的,NaN,一级,de,的
1,2,的,NaN,四级,dí,"的, 的"
2,3,的,NaN,二级,dì,"的, 的, 的"
3,4,是,NaN,一级,shì,"的, 的, 的, 是"
4,5,在,NaN,一级,zài,"的, 的, 的, 是, 在"
...,...,...,...,...,...,...
3144,3145,酗,NaN,高等,xù,"的, 的, 的, 是, 在, 了, 了, 我, 和, 和, 和, 有, 不, 人, 你, 也..."
3145,3146,窿,NaN,高等,lóng,"的, 的, 的, 是, 在, 了, 了, 我, 和, 和, 和, 有, 不, 人, 你, 也..."
3146,3147,屹,NaN,高等,yì,"的, 的, 的, 是, 在, 了, 了, 我, 和, 和, 和, 有, 不, 人, 你, 也..."
3147,3148,朦,NaN,高等,méng,"的, 的, 的, 是, 在, 了, 了, 我, 和, 和, 和, 有, 不, 人, 你, 也..."


In [4]:
def find_words(df, words, num_chars_interval=(2, 10), max_page=-1):
    for page, row in df.iterrows():
        chars = row['old_chars']
        contains = words['word'].apply(lambda x: all(
            char in chars for char in x) and num_chars_interval[0] <= len(x) <= num_chars_interval[1])

        df.at[page, 'words'] = words[contains]['word'].values.tolist()

        # print(
        #     f"Page {row['page']}:\n\tChars: {chars}\n\tWords: {len(df.at[page, 'words'])}\n")

        # stop at max_page
        if row['page'] == max_page:
            break


find_words(df, words, num_chars_interval=(2, 10))

In [ ]:
# Define the order
level_order = ['一级', '二级', '三级', '四级', '五级', '六级', '高等']

# Sort the dataframe
df['level'] = pd.Categorical(df['level'], level_order)
df.sort_values(['level', 'page'], inplace=True)

# Save to csv
df.to_csv('guide.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
# save every level (as numbers) to csv
for i, level in enumerate(level_order, 1):
    df[df['level'] == level].to_csv(
        f'guide_{i}.csv', sep=',', encoding='utf-8', index=False)